In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical ,Sequence
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout, ConvLSTM2D
from tensorflow.keras.optimizers import Adadelta, Nadam ,Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.utils import get_file
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import plot_model
from segmentation_models.losses import CategoricalCELoss
from tensorflow.keras.utils import to_categorical

import os
from glob import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
from pathlib import Path
import shutil
from random import sample, choice, shuffle
import tifffile as tiff
from skimage.transform import rotate
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from sklearn.metrics import classification_report, confusion_matrix
import time
from Patch import Patch

## Generates Patches

In [ ]:
# Extract every patch in a list
monthes = ['202007', '202008', '202009', '202011']
start_time = time.time()
list_patches = Patch.generate_list_patches('/media/wenger/DATA2/dataset_v1/labels')
print("--- %s seconds ---" % (time.time() - start_time))

dates_to_keep_s2 = []
dates_to_keep_s1 = []
list_extracted_patches = []

for e in list_patches:
    days_gap = e.has_days_gap_s2(monthes, date_format='%Y%m', days_gap=17)
    if days_gap[1]:
        e.extracted_s2 = e.reconstruct_filename('s2', days_gap[0])
        list_extracted_patches.append(e)
        dates_to_keep_s2.append(e.extracted_s2)

'''for e in list_patches:
    days_gap = e.has_days_gap_s2(monthes, date_format='%Y%m', days_gap=17)
    if days_gap[1]:
        dates_to_keep_s1.append(e.reconstruct_filename('s1', days_gap[0]))'''

## Extract mean/std

In [ ]:
means_s2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
stds_s2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
means_s1 = [0, 0]
stds_s1 = [0, 0]
patch_list_s2 = dates_to_keep_s2
patch_list_s1 = []
folder_s2 = '/media/wenger/DATA2/dataset_v1/s2'
folder_s1 = '/media/wenger/DATA2/dataset_v1/s1'
nb_s2 = 0

i = 1
_debug_list = patch_list_s2[1:500]
for dates in patch_list_s2:
    for d in dates:
        array = tiff.imread(os.path.join(folder_s2, d)).astype(int)

        for b in range(0, array.shape[2]):
            mean = np.mean(array[:,:,b])
            std = np.std(array[:,:,b])
            means_s2[b] += mean
            stds_s2[b] += std
        nb_s2 += 1
        
    print(str(i) + '/' + str(len(patch_list_s2)))
    i+=1

'''for patch in patch_list_s1:
    array = tiff.imread(os.path.join(folder_s1, patch)).astype(float)
    
    for b in range(0, array.shape[2]):
        mean = np.mean(array[b])
        std = np.std(array[b])
        means_s1[b] += mean
        stds_s1[b] += std'''

means_s2_total = [x / nb_s2 for x in means_s2]
stds_s2_total = [y / nb_s2 for y in stds_s2]
'''means_s1 = means_s1/len(patch_list_s1)
stds_s1 = stds_s1/len(patch_list_s1)'''

In [ ]:
#Means and standard deviation for each 10 bands for 17 days gap for monthes ['202007', '202008', '202009', '202011']
stds_s2_total = [439.62790900199644, 702.2237497462481, 786.5948955924482, 1142.9799955196413, 2129.662793514077,
                2498.619403949213, 2620.957668050562, 2816.7809424162315, 2212.9231966132015, 1352.3410809599484]

means_s2_total = [252.62887057435196, 323.5247227959752, 508.1530443259149, 457.6545855011113, 506.34562773934624,
                628.1464899719209, 691.054464119701, 675.529670439067, 720.6092056255868, 612.9317259184817]

## Compute weights

In [ ]:
folder_gr = '/media/wenger/DATA2/dataset_v1/ground_reference'
classes = np.array([1, 2, 3, 4, 5, 6])
frequency = np.zeros(classes.shape, dtype=np.int64)

for patch in list_extracted_patches:
    array = tiff.imread(os.path.join(folder_gr, patch.reconstruct_filename())).astype(int)

    _tmp = np.zeros((array.shape[0], array.shape[1]), dtype=np.int8)

    for b in range(0, len(classes)):
        if b != (len(classes) - 1):
            _tmp += np.where(array == classes[b], b + 1, 0)
        else:
            _tmp += np.where(array >= classes[b], b + 1, 0)

    (unique, counts) = np.unique(_tmp, return_counts=True)
    _frequencies = np.asarray((unique, counts)).T

    for e in _frequencies:
        for c in range(0, len(classes)):
            if e[0] == classes[c]:
                frequency[c] += e[1]

class_weights = np.array([1 / nr for nr in frequency])
class_weights = (class_weights / np.sum(class_weights) * 1000)
class_weights

In [ ]:
#Class weights for 6 classes for 17 days gap for monthes ['202007', '202008', '202009', '202011']
class_weights = [365.65572762,
                 42.46117546,
                 71.36785131,
                 350.52613946,
                 168.5718001,
                 1.41730604]